---
# Tutoriel 1 - Familiarisation avec le dataset MNIST et création d'un premier réseau de neurones avec PyTorch.
---

<center><img src="./img/mlprocess_3.png" alt="Processus d'apprentissage automatique" width="50%"/></center>

Dans ce tutoriel, nous allons reprendre l'exercice précédent avec les images de chiffres (MNIST) et construire un réseau de neurones simple en utilisant la librairie [PyTorch](https://pytorch.org/) développée par Facebook pour créer des réseaux de neurones plus complexes que les [MLPs de Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html).

Voici les différentes étapes suivies dans ce tutoriel:

1. Chargement et exploration du dataset MNIST.
2. Définition d'un réseau simple.
3. Définition de l'optimiseur.
4. Définition de la fonction de perte et d'une métrique.
5. Boucle d'entraînement:
    1. Piger une "minibatch" (pour la SGD)
    2. Forward Pass
    3. Back propagation
    4. Optimisation
6. Résultats sur l'ensemble de test.

Commençons par charger les modules nécessaires et définir quelques fonctions :


In [1]:
%matplotlib inline
import os
import math
import torch
import numpy as np
from torch import optim, nn
from torchvision.transforms import ToTensor
from torchvision.datasets.mnist import MNIST
from torch.utils.data import DataLoader, random_split

torch.manual_seed(42)
np.random.seed(42)

In [2]:
def load_mnist(download=False, path='./', transform=None):
    """Loads the MNIST dataset.

    :param download: Download the dataset
    :param path: Folder to put the dataset
    :return: The train and test dataset
    """
    train_dataset = MNIST(path, train=True, download=download, transform=transform)
    test_dataset = MNIST(path, train=False, download=download, transform=transform)
    return train_dataset, test_dataset


def load_mnist_with_validation_set(download=False, path='./', train_split=0.8):
    """Loads the MNIST dataset.

    :param download: Download the dataset
    :param path: Folder to put the dataset
    :return: The train, valid and test dataset ready to be ingest in a neural network
    """
    train, test = load_mnist(download, path, transform=ToTensor())
    lengths = [round(train_split*len(train)), round((1.0-train_split)*len(train))]
    train, valid = random_split(train, lengths)
    return train, valid, test

def count_number_of_parameters(net):
    """ Count the number of parameters of a neural net

    :param net: a pytorch neural network
    :return: The number of parameters in the net
    """
    return sum(p.numel() for p in net.parameters() if p.requires_grad)

## 1. Chargement et exploration du dataset MNIST

Analysons plus en détail notre dataset, nos entrées et nos sorties (ou nos x et y).

In [3]:
train, test = load_mnist(download=True)

# Retirons les fichiers inutiles pour optimiser l'espace utilisé.
!rm -f ./MNIST/raw/*ubyte*

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [4]:
train, test

(Dataset MNIST
     Number of datapoints: 60000
     Split: train
     Root Location: ./
     Transforms (if any): None
     Target Transforms (if any): None, Dataset MNIST
     Number of datapoints: 10000
     Split: test
     Root Location: ./
     Transforms (if any): None
     Target Transforms (if any): None)

In [5]:
# Tailles des ensembles d'entraînement et de test
len(train), len(test)

(60000, 10000)

In [6]:
# Affiche la première image du dataset.
first_image = train[0]
print(first_image)
first_image[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x215D81F0198>, tensor(5))


In [7]:
?ToTensor

In [8]:
tensor = ToTensor()(first_image[0])
tensor

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [9]:
# Dimensions des images en tons de gris
tensor.shape

torch.Size([1, 28, 28])

In [10]:
?train

In [11]:
train.transform = ToTensor()
test.transform = ToTensor()

### Création d'un ensemble de validation

On va utiliser 20% des données d'entraînement à cette fin.

In [12]:
lengths = [round(0.8*len(train)), round(0.2*len(train))]
train, valid = random_split(train, lengths)

In [13]:
# Tailles finales des trois ensembles
len(train), len(valid), len(test)

(48000, 12000, 10000)

## 2. Définition d'un réseau simple

In [14]:
# Hyperparamètres d'entraînement
use_gpu = torch.cuda.is_available()
n_epoch = 20
batch_size = 32
learning_rate = 0.01

Le tableau suivant décrit le réseau en couches que nous allons créer. Le code ci-dessous utilise la manière orientée objet de PyTorch pour définir le réseau.

| Type de couche              | Taille de sortie |      # de paramètres   |
|-----------------------------|:----------------:|:----------------------:|
| Input                       |   1x28x28   |              0            |
| Flatten                     |  1\*28\*28  |              0            |
| **Linear avec 10 neurones** |     10      | 28\*28\*10 + 10 = 7 850 |

\# total de paramètres du réseau: 7 850


In [15]:
class Net(nn.Module): # La classe nn.Module représente un réseau de neurones dans PyTorch.
    def __init__(self):
        super(Net, self).__init__()
        # Nous instancions les couches du réseau dans la méthode __init__.
        self.fully_connected = nn.Linear(28 * 28, 10)

    def forward(self, x):
        # La méthode forward spécifie les calculs faits par le réseau.
        x = x.view(-1, 28 * 28)
        x = self.fully_connected(x)
        return x

model = Net()
if use_gpu:
    model.cuda()
model

Net(
  (fully_connected): Linear(in_features=784, out_features=10, bias=True)
)

In [16]:
parameters = list(model.parameters())
len(parameters)

2

In [17]:
parameters[0], parameters[1]

(Parameter containing:
 tensor([[ 0.0110, -0.0096, -0.0194,  ...,  0.0244, -0.0186,  0.0118],
         [-0.0061,  0.0233, -0.0160,  ...,  0.0348, -0.0215,  0.0044],
         [-0.0071,  0.0019,  0.0007,  ...,  0.0242,  0.0239, -0.0050],
         ...,
         [ 0.0130, -0.0074,  0.0242,  ...,  0.0298, -0.0304,  0.0287],
         [ 0.0353,  0.0200, -0.0307,  ..., -0.0224, -0.0333, -0.0140],
         [-0.0011, -0.0336,  0.0070,  ...,  0.0062,  0.0054, -0.0352]],
        requires_grad=True), Parameter containing:
 tensor([ 0.0111, -0.0121,  0.0291,  0.0269, -0.0321,  0.0173,  0.0212, -0.0109,
          0.0141,  0.0218], requires_grad=True))

In [18]:
parameters[0].shape

torch.Size([10, 784])

In [19]:
count_number_of_parameters(model)

7850

## 3. Définition de l'optimiseur

L'optimiseur a besoin des paramètres du réseau à optimiser ainsi que du taux d'apprentissage (`learning rate`) pour calculer les ajustements de poids de neurones à effectuer pendant l'optimisation.

In [20]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

## 4. Définition de la fonction de perte et d'une métrique

NB: Dans PyTorch, la fonction de perte ```CrossEntropyLoss``` implique une couche ```Softmax``` en sortie.

In [21]:
criterion = nn.CrossEntropyLoss()

Pour la métrique, nous allons utiliser l'exactitude fournie par ```sklearn```.

In [22]:
from sklearn.metrics import accuracy_score
def metric(y_true, y_pred): return accuracy_score(y_true, y_pred) * 100

## 5 Boucle d'entraînement

Nous définissons des chargeurs de données (`dataloaders`) qui vont nous fournir `batch_size` images à la fois, c'est-à-dire des lots (`batchs`) de taille `batch_size`. Nous avons 3  chargeurs de données pour les 3 ensembles utilisés: entraînement, validation et test.

In [23]:
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

Nous définissons maintenant une classe Trainer qui aura la tâche d'effectuer l'entraînement et l'évaluation de notre modèle. Dans le prochain tutoriel, nous explorerons la librairie Poutyne qui va faire tout ça pour nous et bien plus encore.

Un des détails à comprendre à lisant le code ci-dessous est que le gradient est stocké dans les paramètres lorsque PyTorch le calcule pour nous. Nous n'avons donc pas besoin d'accéder au gradient directement. L'optimiseur se charge de tout pour nous.


In [24]:
class Trainer:
    def __init__(self, model, optimizer, criterion, metric, use_gpu=False):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.metric = metric
        self.use_gpu = use_gpu
        
    def evaluate_model(self, loader):
        true = []
        pred = []
        val_loss = []

        # Activation du mode d'évaluation du modèle
        self.model.eval()
        
        # Désactivation du calcul du gradient en évaluation
        with torch.no_grad():
            for batch in loader:
                inputs, targets = batch
                
                # Envoi de la batch sur les GPUs si disponibles.
                if self.use_gpu:
                    inputs = inputs.cuda()
                    targets = targets.cuda()

                # Appel de la méthode forward de notre modèle.
                output = self.model(inputs)

                predictions = output.max(dim=1)[1]

                val_loss.append(self.criterion(output, targets).item())
                true.extend(targets.detach().cpu().numpy().tolist())
                pred.extend(predictions.detach().cpu().numpy().tolist())

        return self.metric(true, pred), sum(val_loss) / len(val_loss)

    def train_model(self, train_loader, valid_loader, n_epoch):
        for i in range(n_epoch):
            # Activation du mode d'entraînement du modèle
            self.model.train()
            
            for batch in train_loader:
                inputs, targets = batch
                
                # Envoi de la batch sur les GPUs si disponibles.
                if self.use_gpu:
                    inputs = inputs.cuda()
                    targets = targets.cuda()
                
                # Le gradient est mis à zéro pour éviter que le gradient de l'ancienne et de la nouvelle batch se mélangent. 
                self.optimizer.zero_grad()
                
                # Appel de la méthode forward de notre modèle.
                output = self.model(inputs)

                # Calcul de la perte
                loss = self.criterion(output, targets)
                
                # Rétropropagation du gradient (le gradient est stocké dans les paramètres)
                loss.backward()
                
                # Mise à jour des poids en fonction du gradient calculé.
                self.optimizer.step()

            train_acc, train_loss = self.evaluate_model(train_loader)
            val_acc, val_loss = self.evaluate_model(valid_loader)
            
            print('Epoch {} - Train acc: {:.2f} - Val acc: {:.2f} - Train loss: {:.4f} - Val loss: {:.4f}'.format(
                i,
                train_acc,
                val_acc,
                train_loss,
                val_loss
            ))

On peut maintenant entraîner et de tester notre modèle! Le réseau sera entraîné sur 20 époques tel que spécifié plus haut dans 
les hyperparamètres d'entraînement.

In [25]:
trainer = Trainer(model, optimizer, criterion, metric, use_gpu)
trainer.train_model(train_loader, valid_loader, n_epoch)

Epoch 0 - Train acc: 87.21 - Val acc: 86.83 - Train loss: 0.5331 - Val loss: 0.5452
Epoch 1 - Train acc: 88.61 - Val acc: 88.08 - Train loss: 0.4393 - Val loss: 0.4530
Epoch 2 - Train acc: 89.31 - Val acc: 88.86 - Train loss: 0.4002 - Val loss: 0.4158
Epoch 3 - Train acc: 89.82 - Val acc: 89.26 - Train loss: 0.3770 - Val loss: 0.3933
Epoch 4 - Train acc: 90.14 - Val acc: 89.64 - Train loss: 0.3617 - Val loss: 0.3794
Epoch 5 - Train acc: 90.42 - Val acc: 89.83 - Train loss: 0.3502 - Val loss: 0.3689
Epoch 6 - Train acc: 90.61 - Val acc: 90.03 - Train loss: 0.3414 - Val loss: 0.3609
Epoch 7 - Train acc: 90.74 - Val acc: 90.10 - Train loss: 0.3340 - Val loss: 0.3542
Epoch 8 - Train acc: 90.91 - Val acc: 90.22 - Train loss: 0.3280 - Val loss: 0.3490
Epoch 9 - Train acc: 90.99 - Val acc: 90.33 - Train loss: 0.3237 - Val loss: 0.3451
Epoch 10 - Train acc: 91.19 - Val acc: 90.45 - Train loss: 0.3189 - Val loss: 0.3418
Epoch 11 - Train acc: 91.26 - Val acc: 90.48 - Train loss: 0.3150 - Val los

In [26]:
test_acc, test_loss = trainer.evaluate_model(test_loader)

In [27]:
# Performances finales en test
test_acc, test_loss

(91.77, 0.29140586808276253)